# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [ ]:
import json
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import folium
from markdown import markdown

In [ ]:
# Your code here

#Our previous function for loading our api key file
def get_keys(path,api=None):
    with open(path) as f:
        if api:
            return json.load(f)[api]
        else:
            return json.load(f)

keys = get_keys(os.environ['HOME']+"/.secret/keys.json",'yelp')

api_key = keys['api_key']

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'location': 'DC',
                'limit' : 50,
                'categories': 'bookstores,usedbooks'
            }
response = requests.get(url, headers=headers, params=url_params)
response.text

In [ ]:
response.json()['total']

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

limit = 50
url_params = {
                'location': 'DC',
                'limit' : limit,
                'categories': 'bookstores'
            }

businesses = response.json()['businesses']
total = response.json()['total']
offset = 0
max_request = max(total,4999)
attempts = 0
max_attempts = total // limit + 1
while (len(businesses) < max_request) and (attempts < max_attempts):
    url_params['offset'] = len(businesses)
    response = requests.get(url, headers=headers, params=url_params)    
    businesses.extend(response.json()['businesses'])
    attempts += 1

    time.sleep(1)


Let's use pandas' json_normalize functionality.

In [ ]:
df = pd.io.json.json_normalize(businesses).set_index('id')
df.head()

In [ ]:
df['price_val'] = df.price.map(len,na_action='ignore')
df.head()

We can generate a side dataframe with the categories broken out.

In [ ]:
cat_df = pd.io.json.json_normalize(businesses, 'categories',['id']).rename(columns={'id':'business_id'})
cat_df

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
df.describe()

In [ ]:
pd.plotting.scatter_matrix(df[['distance','rating','review_count','price_val']]);

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
base_map = folium.Map([df['coordinates.latitude'].mean(), df['coordinates.longitude'].mean()], zoom_start=13)
base_map

In [ ]:
df.rating.hist()

In [ ]:
df['rating_bin'] = pd.cut(df.rating, 6, labels=False, retbins=False, right=True)
df.head()

In [ ]:
colors = [ 'darkred', 'red', 'orange', 'lightblue', 'blue',  'darkblue']

base_map = folium.Map([df['coordinates.latitude'].mean(), df['coordinates.longitude'].mean()], zoom_start=13)


def row_to_marker(row, map = base_map):
    lat = row['coordinates.latitude']
    long = row['coordinates.longitude']
    color = colors[row.rating_bin]
    popup_text = f"**{row['name']}**  \nrating: {row.rating}"
    if not pd.isna(row['price']):
        popup_text += f"  \nprice: {row['price']}"
    icon = folium.Icon(icon='map-marker-alt', color=color)
    folium.Marker(location=[lat, long], popup=markdown(popup_text), icon=icon).add_to(base_map)
    
df.apply(row_to_marker, axis=1)
base_map

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!